In [3]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%autoreload 2

In [5]:
prompt = """Look at this food image and provide a recipe in XML format.
    Use the following structure:
    <recipe>
      <title>The dish name</title>
      <ingredients>
        <ingredient>1 cup whole milk</ingredient>
        <ingredient>2 tbsp sugar</ingredient>
        <!-- Add all ingredients in this format -->
      </ingredients>
      <instructions>
        <step>1. Preheat the oven to 350°F.</step>
        <step>2. Mix all ingredients in a bowl.</step>
        <!-- Add all steps in this format -->
      </instructions>
    </recipe>
    
    Important: 
    1. Make sure to format your response as valid XML following the exact structure above.
    2. For each ingredient, include quantity, unit, and name in the <ingredient> tag.
    3. For each instruction, include the step number and description in the <step> tag.
    4. Do not use commas to separate ingredients - use individual <ingredient> tags.
    5. Do not use newlines or commas inside the instruction text.
    """

In [6]:
%pip install pandas datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install fuzzywuzzy python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from datasets import Dataset
import os


# File paths
csv_file_path = "inverse_cooking_dataset/Food Ingredients and Recipe Dataset with Image Name Mapping.csv"
image_dir_path = "inverse_cooking_dataset/Food Images/Food Images"

# Load the CSV file directly
print(f"Loading CSV from: {csv_file_path}")
df = pd.read_csv(csv_file_path)

# Add the full image paths
if 'Image_Name' in df.columns:
    print(f"Adding image paths from: {image_dir_path}")
    df['full_image_path'] = df['Image_Name'].apply(lambda x: os.path.join(image_dir_path, f"{x}.jpg"))
    
    # Verify a few paths (optional)
    print(f"First few image paths: {df['full_image_path'].head(2).tolist()}")

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)
print(f"Dataset created with {len(hf_dataset)} examples")
print(f"Sample data: {hf_dataset[0]}")

/Users/juyounglee/Desktop/School/2025Spring/csci467/inverse_cooking/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading CSV from: inverse_cooking_dataset/Food Ingredients and Recipe Dataset with Image Name Mapping.csv
Adding image paths from: inverse_cooking_dataset/Food Images/Food Images
First few image paths: ['inverse_cooking_dataset/Food Images/Food Images/miso-butter-roast-chicken-acorn-squash-panzanella.jpg', 'inverse_cooking_dataset/Food Images/Food Images/crispy-salt-and-pepper-potatoes-dan-kluger.jpg']
Dataset created with 13501 examples
Sample data: {'Unnamed: 0': 0, 'Title': 'Miso-Butter Roast Chicken With Acorn Squash Panzanella', 'Ingredients': '[\'1 (3½–4-lb.) whole chicken\', \'2¾ tsp. kosher salt, divided, plus more\', \'2 small acorn squash (about 3 lb. total)\', \'2 Tbsp. finely chopped sage\', \'1 Tbsp. finely chopped rosemary\', \'6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature\', \'¼ tsp. ground allspice\', \'Pinch of crushed red pepper flakes\', \'Freshly ground black pepper\', \'⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)\', 

In [9]:
from utils import preprocess_dataset

preprocessed_dataset = preprocess_dataset(hf_dataset.select(range(10)))

Preprocessing dataset with 10 examples...


Filter: 100%|██████████| 10/10 [00:00<00:00, 756.37 examples/s]

Preprocessing complete. 10 examples with valid images (filtered out 0 examples)


In [10]:
preprocessed_dataset.features


{'Unnamed: 0': Value(dtype='int64', id=None),
 'Title': Value(dtype='string', id=None),
 'Ingredients': Value(dtype='string', id=None),
 'Instructions': Value(dtype='string', id=None),
 'Image_Name': Value(dtype='string', id=None),
 'Cleaned_Ingredients': Value(dtype='string', id=None),
 'full_image_path': Value(dtype='string', id=None),
 'parsed_ingredients': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'parsed_cleaned_ingredients': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'instruction_steps': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'base64_image': Value(dtype='string', id=None),
 'ingredients_embeddings': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None),
 'instructions_embeddings': Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None)}

In [11]:
preprocessed_dataset.to_pandas().head()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients,full_image_path,parsed_ingredients,parsed_cleaned_ingredients,instruction_steps,base64_image,ingredients_embeddings,instructions_embeddings
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...",inverse_cooking_dataset/Food Images/Food Image...,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","[Pat chicken dry with paper towels, season all...",/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...,"[[0.040941693, 0.011311965, -0.043800976, 0.01...","[[-0.065366134, -0.03621227, 0.086215384, 0.00..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (...",inverse_cooking_dataset/Food Images/Food Image...,"[2 large egg whites, 1 pound new potatoes (abo...","[2 large egg whites, 1 pound new potatoes (abo...",[Preheat oven to 400°F and line a rimmed bakin...,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...,"[[0.044729907, 0.047785446, 0.008685435, -0.00...","[[0.082553715, -0.013618565, -0.06954032, -0.0..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ...",inverse_cooking_dataset/Food Images/Food Image...,"[1 cup evaporated milk, 1 cup whole milk, 1 ts...","[1 cup evaporated milk, 1 cup whole milk, 1 ts...",[Place a rack in middle of oven; preheat to 40...,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...,"[[-0.012475215, -0.033795163, 0.017434442, 0.0...","[[-0.019274333, 0.0040179007, -0.008754797, -0..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in...",inverse_cooking_dataset/Food Images/Food Image...,"[1 (¾- to 1-pound) round Italian loaf, cut int...","[1 (¾- to 1-pound) round Italian loaf, cut int...",[Preheat oven to 350°F with rack in middle. Ge...,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...,"[[-0.0077589653, 0.07906025, -0.029666271, 0.0...","[[0.05118617, 0.038572542, -0.050694022, -0.03..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",inverse_cooking_dataset/Food Images/Food Image...,"[1 teaspoon dark brown sugar, 1 teaspoon hot w...","[1 teaspoon dark brown sugar, 1 teaspoon hot w...",[Stir together brown sugar and hot water in a ...,/9j/4AAQSkZJRgABAQAAAQABAAD/4gIcSUNDX1BST0ZJTE...,"[[-0.07132222, -0.044993084, -0.037271444, -0....","[[-0.061055828, -0.027506301, 0.004912753, 0.0..."


In [12]:
test_dataset = preprocessed_dataset.select(range(5))

In [13]:
# generate recipe for each example
from utils import generate_response, parse_recipe_xml
responses = []
for example in test_dataset:
    input_image = example['base64_image']
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{input_image}",
                    },
                },
            ],
        }
    ]
    response = generate_response(messages)
    responses.append(response)


In [14]:
y_pred = []
for response in responses:
    parsed_recipe = parse_recipe_xml(response)
    y_pred.append(parsed_recipe)

In [15]:
y_pred

[{'title': 'Roasted Whole Chicken with Gravy and Roasted Vegetables',
  'ingredients': ['1 whole chicken (3-4 lbs)',
   '2 tbsp olive oil',
   '1 tsp salt',
   '1 tsp pepper',
   '1 tsp dried thyme',
   '1 tsp dried rosemary',
   '1 large onion, peeled and chopped',
   '2 cloves garlic, minced',
   '2 carrots, peeled and chopped',
   '2 celery stalks, chopped',
   '2 cups mixed roasted vegetables (such as Brussels sprouts, red bell peppers, and red onions)',
   '1 cup whole milk',
   '2 tbsp all-purpose flour',
   '2 cups chicken broth',
   '1 cup heavy cream',
   '2 tbsp butter',
   '1 tsp Dijon mustard'],
  'steps': ['1. Preheat the oven to 425°F.',
   '2. Rinse the chicken and pat it dry with paper towels.',
   '3. In a small bowl, mix together the olive oil, salt, pepper, thyme, and rosemary.',
   '4. Rub the mixture all over the chicken, making sure to get some under the skin as well.',
   '5. Place the chopped onion, garlic, carrots, and celery in a large roasting pan.',
   '6. P

In [16]:
for i in range(len(y_pred)):
    print(f"Example {i+1}:")
    print(f"Title: {y_pred[i]['title']}")
    print("Ingredients:")
    for ingredient in y_pred[i]['ingredients']:
        print(f"- {ingredient}")
    print("Instructions:")
    for step in y_pred[i]['steps']:
        print(f"{step}")

Example 1:
Title: Roasted Whole Chicken with Gravy and Roasted Vegetables
Ingredients:
- 1 whole chicken (3-4 lbs)
- 2 tbsp olive oil
- 1 tsp salt
- 1 tsp pepper
- 1 tsp dried thyme
- 1 tsp dried rosemary
- 1 large onion, peeled and chopped
- 2 cloves garlic, minced
- 2 carrots, peeled and chopped
- 2 celery stalks, chopped
- 2 cups mixed roasted vegetables (such as Brussels sprouts, red bell peppers, and red onions)
- 1 cup whole milk
- 2 tbsp all-purpose flour
- 2 cups chicken broth
- 1 cup heavy cream
- 2 tbsp butter
- 1 tsp Dijon mustard
Instructions:
1. Preheat the oven to 425°F.
2. Rinse the chicken and pat it dry with paper towels.
3. In a small bowl, mix together the olive oil, salt, pepper, thyme, and rosemary.
4. Rub the mixture all over the chicken, making sure to get some under the skin as well.
5. Place the chopped onion, garlic, carrots, and celery in a large roasting pan.
6. Place the chicken on top of the vegetables and roast for 45 minutes.
7. After 45 minutes, add the

In [17]:
%pip install nltk rouge-score sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [18]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juyounglee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/juyounglee/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [19]:
from evals import compute_evals
result = []

for i in range(len(y_pred)):
    pred_recipe = y_pred[i]
    golden_recipe = test_dataset[i]
    result.append(compute_evals(pred_recipe, golden_recipe))
result

Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Evaluation metrics calculated.
Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Evaluation metrics calculated.
Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Evaluation metrics calculated.
Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Evaluation metrics calculated.
Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Evaluation metrics calculated.


[{'cosine_similarity': {'steps': np.float64(0.548320903865328),
   'ingredients': np.float64(0.6695701627852335)},
  'bleu_score': {'steps': 1.1734813596686042e-09,
   'ingredients': 0.07705894643402769},
  'rouge_scores': {'steps': {'rouge1': 0.04908697434108637,
    'rougeL': 0.03588930434355002},
   'ingredients': {'rouge1': 0.4654613360495714,
    'rougeL': 0.44585349291231646}}},
 {'cosine_similarity': {'steps': np.float64(0.46956712630995917),
   'ingredients': np.float64(0.5601576382879483)},
  'bleu_score': {'steps': 0.00012416267630336477,
   'ingredients': 0.03630476071789092},
  'rouge_scores': {'steps': {'rouge1': 0.08824544142659338,
    'rougeL': 0.07459842678567052},
   'ingredients': {'rouge1': 0.30466914038342613,
    'rougeL': 0.30466914038342613}}},
 {'cosine_similarity': {'steps': np.float64(0.5576354185838021),
   'ingredients': np.float64(0.6523539056836594)},
  'bleu_score': {'steps': 1.590108466702062e-05,
   'ingredients': 0.1366271236366746},
  'rouge_scores':

In [22]:
for i in range(len(result)):
    print(f"Example {i+1}:")
    print(f"Instruction Cosine Similarity: {result[i]['cosine_similarity']['steps']}")
    print(f"Ingredient Cosine Similarity: {result[i]['cosine_similarity']['ingredients']}")
    print(f"Instruction BLEU Score: {result[i]['bleu_score']['steps']}")
    print(f"Ingredient BLEU Score: {result[i]['bleu_score']['ingredients']}")
    print(f"Instruction ROUGE-1 Score: {result[i]['rouge_scores']['steps']['rouge1']}")
    print(f"Instruction ROUGE-L Score: {result[i]['rouge_scores']['steps']['rougeL']}")
    print(f"Ingredient ROUGE-1 Score: {result[i]['rouge_scores']['ingredients']['rouge1']}")
    print(f"Ingredient ROUGE-L Score: {result[i]['rouge_scores']['ingredients']['rougeL']}")
    print("---------------------------------------------")


Example 1:
Instruction Cosine Similarity: 0.548320903865328
Ingredient Cosine Similarity: 0.6695701627852335
Instruction BLEU Score: 1.1734813596686042e-09
Ingredient BLEU Score: 0.07705894643402769
Instruction ROUGE-1 Score: 0.04908697434108637
Instruction ROUGE-L Score: 0.03588930434355002
Ingredient ROUGE-1 Score: 0.4654613360495714
Ingredient ROUGE-L Score: 0.44585349291231646
---------------------------------------------
Example 2:
Instruction Cosine Similarity: 0.46956712630995917
Ingredient Cosine Similarity: 0.5601576382879483
Instruction BLEU Score: 0.00012416267630336477
Ingredient BLEU Score: 0.03630476071789092
Instruction ROUGE-1 Score: 0.08824544142659338
Instruction ROUGE-L Score: 0.07459842678567052
Ingredient ROUGE-1 Score: 0.30466914038342613
Ingredient ROUGE-L Score: 0.30466914038342613
---------------------------------------------
Example 3:
Instruction Cosine Similarity: 0.5576354185838021
Ingredient Cosine Similarity: 0.6523539056836594
Instruction BLEU Score: 1.5

In [32]:
#pretty print generated recipe and golden recipe
def pretty_print_recipe(generated_recipe, golden_recipe):
    print("--- Generated Recipe ---")
    print(f"Title: {generated_recipe.get('title', 'N/A')}") # Use .get for safety

    print("Ingredients:")
    if 'ingredients' in generated_recipe and generated_recipe['ingredients']:
        for ingredient in generated_recipe['ingredients']:
            print(f"- {ingredient}")
    else:
        print("- (No ingredients generated)")

    print("Instructions:")
    if 'steps' in generated_recipe and generated_recipe['steps']:
        for step in generated_recipe['steps']:
            print(f"{step}") # Assuming steps are numbered in the generated output
    else:
        print("- (No instructions generated)")

    print("---------------------------------------------")
    print("--- Golden Recipe ---")
    print(f"Title: {golden_recipe.get('Title', 'N/A')}") # Use .get for safety

    print("Ingredients:")
    # Use 'parsed_ingredients' for the golden recipe list
    if 'parsed_ingredients' in golden_recipe and golden_recipe['parsed_ingredients']:
        for ingredient in golden_recipe['parsed_ingredients']:
            print(f"- {ingredient}")
    else:
        print("- (No ingredients in golden data)")

    print("Instructions:")
    # Use 'instruction_steps' for the golden recipe list
    if 'instruction_steps' in golden_recipe and golden_recipe['instruction_steps']:
        for step in golden_recipe['instruction_steps']:
            print(f"{step}") # Golden steps are likely just text, not numbered like "1. "
    else:
        print("- (No instructions in golden data)")

    print("=============================================") # Separator between examples

In [33]:
for i in range(len(y_pred)):
    pretty_print_recipe(y_pred[i], test_dataset[i])

--- Generated Recipe ---
Title: Roasted Whole Chicken with Gravy and Roasted Vegetables
Ingredients:
- 1 whole chicken (3-4 lbs)
- 2 tbsp olive oil
- 1 tsp salt
- 1 tsp pepper
- 1 tsp dried thyme
- 1 tsp dried rosemary
- 1 large onion, peeled and chopped
- 2 cloves garlic, minced
- 2 carrots, peeled and chopped
- 2 celery stalks, chopped
- 2 cups mixed roasted vegetables (such as Brussels sprouts, red bell peppers, and red onions)
- 1 cup whole milk
- 2 tbsp all-purpose flour
- 2 cups chicken broth
- 1 cup heavy cream
- 2 tbsp butter
- 1 tsp Dijon mustard
Instructions:
1. Preheat the oven to 425°F.
2. Rinse the chicken and pat it dry with paper towels.
3. In a small bowl, mix together the olive oil, salt, pepper, thyme, and rosemary.
4. Rub the mixture all over the chicken, making sure to get some under the skin as well.
5. Place the chopped onion, garlic, carrots, and celery in a large roasting pan.
6. Place the chicken on top of the vegetables and roast for 45 minutes.
7. After 45 mi